# Fetching Gene and Associated Variant Data from EMBL

## Setup


In [1]:
import sys
from pathlib import Path

import requests
import polars as pl
import os
from collections import defaultdict
import time
import plotly.io as pio

# Import src directory
NOTEBOOK_DIR = Path.cwd()
SRC = NOTEBOOK_DIR.parent / "src"
if str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

from helpers import chunks

# Required for `nbconvert` to include Plotly plots in the output HTML
pio.renderers.default = "notebook"

/Users/lukasschonmann/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Parameter Definition


In [2]:
# Either the symbold or Ensembl gene ID to analyze
GENE_ID = "CYP2D6"

os.makedirs("../data/processed", exist_ok=True)

## Query EMBL Database for IDs

This is helper code to look up an external symbol (e.g. a display name for a gene/transcript, a synonym, or an externally linked reference). It returns an EMBL ID. 


In [3]:
symbol = "CYP2D6"
species = "human"

BASE = "https://rest.ensembl.org/"
headers = {"Content-Type": "application/json"}
endpoint = "xrefs/symbol/" + species + "/" + symbol

id = "ENSG00000139618"
params = {}

url = BASE + endpoint

r = requests.get(url, params=params, headers=headers)
try:
    r.raise_for_status()
except requests.exceptions.HTTPError as e:
    print("API Error:", e)
stats = r.json()

if stats:
    print(f"Found {len(stats)} entries for symbol '{symbol}' in species '{species}':")
    for entry in sorted(stats, key=lambda x: x["id"]):
        print(f"ID: {entry['id']}, Type: {entry['type']}")
else:
    print(f"No entries found for symbol '{symbol}' in species '{species}'.")

Found 8 entries for symbol 'CYP2D6' in species 'human':
ID: ENSG00000100197, Type: gene
ID: ENSG00000272000, Type: gene
ID: ENSG00000272532, Type: gene
ID: ENSG00000275211, Type: gene
ID: ENSG00000280905, Type: gene
ID: ENSG00000282966, Type: gene
ID: ENSG00000283284, Type: gene
ID: LRG_303, Type: gene


## Get Gene Region Based on a EMBL Stable ID or Gene Symbol

Versatile wrapper function that searches the EMBL database for genes either based on an EMBL stable ID or a gene symbol + species. 

If the ID does not start with "ENSG" search must be set explicitly to IDs with `is_id=True`. 

Prints the most important information and returns the fetched object. 

In [4]:
def embl_lookup(name, species="human", is_id=False, verbose=True):
    # Determine if the input is an EMBL ID or a gene symbol
    if is_id or name.startswith("ENSG"):
        type = "id"
        species = ""
    else:
        type = "symbol"
        species = species + "/"
    BASE = "https://rest.ensembl.org/"
    headers = {"Content-Type": "application/json"}
    endpoint = "lookup/" + type + "/"

    id = "ENSG00000139618"
    params = {}

    url = BASE + endpoint + species + name

    r = requests.get(url, params=params, headers=headers)
    try:
        r.raise_for_status()
    except requests.exceptions.HTTPError as e:
        print("Gene not found.")
        print("API Error:", e)
        return
    stats = r.json()

    if verbose:
        print(f"Gene name: {stats.get('display_name')}")
        print(f"Gene ID: {stats.get('id')}")
        print(f"Description: {stats.get('description')}")
        print(f"Species: {stats.get('species')}")
        print(f"Assembly name: {stats.get('assembly_name')}")
        print(f"Chromosome: {stats.get('seq_region_name')}")
        print(f"Start: {stats.get('start')}")
        print(f"End: {stats.get('end')}")

        # Calculate gene length in kilobase pairs (kbp) and format to 2 decimal places
        gene_length = stats.get("end") - stats.get("start") + 1
        gene_length_kbp = f"{gene_length / 1000:.2f}"
        print(f"Gene length: {gene_length_kbp} kbp")

    return stats


gene_data = embl_lookup(GENE_ID, species="human", is_id=False)

Gene name: CYP2D6
Gene ID: ENSG00000100197
Description: cytochrome P450 family 2 subfamily D member 6 (gene/pseudogene) [Source:HGNC Symbol;Acc:HGNC:2625]
Species: human
Assembly name: GRCh38
Chromosome: 22
Start: 42125962
End: 42131236
Gene length: 5.28 kbp


## Fetch Variants for a Gene

In [5]:
def fetch_variants(name, species="human", is_id=False, verbose=True):
    gene_data = embl_lookup(name, species=species, is_id=is_id, verbose=verbose)
    if not gene_data:
        return

    region = f"{gene_data['seq_region_name']}:{gene_data['start']}-{gene_data['end']}"
    params = {"feature": "variation"}

    BASE = "https://rest.ensembl.org/"
    headers = {"Content-Type": "application/json"}
    endpoint = "overlap/region/" + species + "/" + region

    url = BASE + endpoint

    r = requests.get(url, params=params, headers=headers)
    try:
        r.raise_for_status()
    except requests.exceptions.HTTPError as e:
        print("Unable to fetch variants.")
        print("API Error:", e)
        return None
    stats = r.json()

    return stats


variants = fetch_variants(name=GENE_ID, verbose=False)
df_vars = pl.DataFrame(variants)

### Drop Multiallelic Variants and Transform Columns

The table will be saved as a Parquet file. 

In [6]:
# Drop multiallelic variants and special cases
count_variants_all = df_vars.height
df_vars = df_vars.filter(pl.col("alleles").list.len() == 2)
count_variants_biallelic = df_vars.height

print(
    f"Dropped {count_variants_all - count_variants_biallelic} multiallelic or special variants."
)
print(f"Remaining biallelic variants: {count_variants_biallelic}.")

# Select and rename relevant columns
df_vars = df_vars.select(
    [
        pl.col("id"),
        pl.col("seq_region_name").alias("chr"),
        pl.col("start").alias("pos"),
        pl.col("alleles").list.get(0).alias("ref"),
        pl.col("alleles").list.get(1).alias("alt"),
        pl.col("consequence_type").alias("consequence"),
        pl.col("clinical_significance"),
    ]
)

# Write to Parquet file
out_path = f"../data/processed/{GENE_ID}-variants.parquet"
df_vars.write_parquet(out_path)
print(f"Saved variant data to '{out_path}'.")

Dropped 1331 multiallelic or special variants.
Remaining biallelic variants: 2451.
Saved variant data to '../data/processed/CYP2D6-variants.parquet'.


### Add Population Variant Details to Table

Fetches the variant minor allele frequencies from EMBL for all populations and merges it with the main table information. This is rate-limited and can take a while (~20 minutes for a standard-sized gene). 

The long format table will be saved as a Parquet file. 

In [7]:
def fetch_variant_frequencies(variant_ids, species="human", batch_size=200, min_interval=0.25):
    """ "
    Fetch variant frequencies from the Ensembl REST API for a given gene.
    The maximum number of variants that can be queried at once is 200.
    """
    assert batch_size <= 200, "Batch size cannot exceed 200."

    params = {"pops": True, "phenotypes": True}

    BASE = "https://rest.ensembl.org/"
    headers = {"Content-Type": "application/json", "Accept": "application/json"}
    endpoint = "variation/" + species

    url = BASE + endpoint

    size_df = len(variant_ids)

    last_request_time = 0.0
    collected_vars = {}
    for i, batch in enumerate(chunks(variant_ids, batch_size), start=1):
        current_progress = min(i * batch_size, size_df)
        print(f"\rProgress: {current_progress}/{size_df} variants", end="", flush=True)
        # Rate limiting requests
        elapsed_time = time.time() - last_request_time
        if elapsed_time < min_interval:
            time.sleep(min_interval - elapsed_time)

        r = requests.post(url, params=params, headers=headers, json={"ids": batch})
        last_request_time = time.time()

        try:
            r.raise_for_status()
        except requests.exceptions.HTTPError as e:
            print("Unable to fetch variants.")
            print("API Error:", e)
            return None

        collected_vars.update(r.json())

    return collected_vars


variants = fetch_variant_frequencies(variant_ids=df_vars["id"].to_list())

# Convert fetched population frequencies into long format
pop_vars = defaultdict(list)
for var_id, var_data in variants.items():
    f_minor_allele = var_data.get("minor_allele", None)
    f_most_severe_consequence = var_data.get("most_severe_consequence", None)
    if "populations" in var_data:
        for pop in var_data["populations"]:
            pop_vars["id"].append(var_id)
            pop_vars["population"].append(pop["population"])
            pop_vars["minor_allele"].append(f_minor_allele)
            pop_vars["MAF"].append(pop["frequency"])
            pop_vars["most_severe_consequence"].append(f_most_severe_consequence)

df_var_freqs = pl.DataFrame(pop_vars, strict=False)

# Join long format frequencies to variant data
df_merged = df_vars.join(df_var_freqs, on="id", how="left")
display(df_merged.head())

# Write to Parquet file
out_path = f"../data/processed/{GENE_ID}-pop-vars.parquet"
df_merged.write_parquet(out_path)
print(f"Saved variant data to '{out_path}'.")

Progress: 2451/2451 variants

id,chr,pos,ref,alt,consequence,clinical_significance,population,minor_allele,MAF,most_severe_consequence
str,str,i64,str,str,str,list[str],str,str,f64,str
"""rs1219101811""","""22""",42125963,"""G""","""A""","""3_prime_UTR_variant""",[],"""gnomADg:fin""",null,1.0,"""splice_polypyrimidine_tract_va…"
"""rs1219101811""","""22""",42125963,"""G""","""A""","""3_prime_UTR_variant""",[],"""gnomADg:nfe""",null,0.000015,"""splice_polypyrimidine_tract_va…"
"""rs1219101811""","""22""",42125963,"""G""","""A""","""3_prime_UTR_variant""",[],"""gnomADg:nfe""",null,1.0,"""splice_polypyrimidine_tract_va…"
"""rs1219101811""","""22""",42125963,"""G""","""A""","""3_prime_UTR_variant""",[],"""gnomADg:mid""",null,1.0,"""splice_polypyrimidine_tract_va…"
"""rs1219101811""","""22""",42125963,"""G""","""A""","""3_prime_UTR_variant""",[],"""gnomADg:amr""",null,1.0,"""splice_polypyrimidine_tract_va…"


Saved variant data to '../data/processed/CYP2D6-pop-vars.parquet'.
